# DIP opdracht 2: Letterfrequenties
Lucas Siekkötter, 1770991 <br>
& <br>
Wijnand van Dijk, 1762733 

Dit is een notebook om onze code te demonstreren en te illustreren. We laten alle functies zien en relevante resultaten om te laten zien dat de gewenste resultaten aanwezig zijn. We kijken per file naar de functies.

## Imports

In [1]:
import numpy
import string

## Code

### info.py
Maakt een lijst met alle leestekens, die lijst kunnen we later gebruiken om makkelijk alle leesteken te vervangen. Ook wordt er een lijst aangemaakt van ons 'alfabet' die bestaat uit het normale alfabet plus '_' en '%'.

In [2]:
leestekens = [".", ",", ":", ";", "\'", "\"", "-", "_", "!", "?", "(", ")", "\\"]

abc = list(string.ascii_lowercase)
abc.extend(["_", "%"])

### mapper.py
De functie `mapper_get_lines()` lijst de hele tekst uit. 

In [3]:
def mapper_get_lines(path):
    """
    Converts .txt to strings and puts it in a list
    
    @param path: path to text data
    @return    : list
    """
    res = []
    with open("data/" + path + ".txt", "r", encoding="utf8") as f:
        lines = f.readlines()

        for line in lines:
            line = line.strip()
            if line != "":
                res.append(line)
    
    return res

`mapper_replace_punct()` zorgt ervoor dat alle leestekens met '%' worden vervanfen, en spaties met '_'

In [4]:
def mapper_replace_punct(text, sign):
    """
    Replace all punctuation with % and replace spaces with _
    
    @param  sign: leestekens
    @return     : text waar leestekens vervangen zijn
    """
    res = []
    for line in text:
        for i in sign:
            line = line.replace(i, "%")
        
        line = line.replace(" ", "_")
        line = line.lower()
        res.append(line)

    return res

### filter.py
`filter_training()` is een filter functie die is bedoeld voor de trainings fase. De functie kijkt bij alle letters in een tekst welke letter of leesteken volgt. Combinaties van leesteken en spatie halen we eruit, omdat dat volgens ons niks zegt over de taal van een zin.


In [5]:
def filter_training(txt):
    """
    Get all 2 letter combinations in text
    
    @param  txt: tekst waat combinaties uit gehaald worden
    @return    : lijst met combinaties in tekst
    """
    combinations = []
    for line in txt:
        for letter in range(0, len(line)):  
            try:
                res = line[letter] + line[letter + 1]
                # leesteken + spatie is niet relevant dus die combinaties kunnen eruit. "\n" zitten er ook tussen en die kunnen ook weg
                # en check of beide letters wel in het (aangepaste) alfabet zitten
                if (res[0] in abc and res[-1] in abc) and (res != "%_" and res != "_%" and "\n" not in res):
                    combinations.append([res])
            except:
                continue
    
    return combinations

`filter_validation()` is de functie voor de validatie fase. De reden dat er twee functies zijn gemaakt die bijna hetzelfde doen is dat de vorm van input tijdens de training- en validatiefase anders is. Dat komt omdat tijdens de validatie de tekst per line is ingedeeld, en tijdens training is dat niet zo.

In [6]:
def filter_validation(txt):
    """
    Get all 2 letter combinations in text.
    
    Different funtion was made to return in correct form
    
    @param  txt: tekst waat combinaties uit gehaald worden
    @return    : lijst met combinaties in tekst
    """
    combinations = []
    for line in txt:
        temp = []
        for letter in range(0, len(line)):  
            try:
                res = line[letter] + line[letter + 1]
                # leesteken + spatie is niet relevant dus die combinaties kunnen eruit. "\n" zitten er ook tussen en die kunnen ook weg
                # en check of beide letters wel in het (aangepaste) alfabet zitten
                if (res[0] in abc and res[-1] in abc) and (res != "%_" and res != "_%" and "\n" not in res):
                    temp.append(res)
            except:
                continue
        combinations.append(temp)
    
    return combinations

### reducer.py

We hebben onze methode gebaseerd op een comment uit de volgende blogpost: https://stackoverflow.com/a/18197422. De comment in kwestie is: 


```
'Take the Wikipedia in English. Check what is the probability that after the letter 'a' comes a 'b' (for example) and do that for all the combination of letters, you will end up with a matrix of probabilities.

If you do the same for the Wikipedia in different languages you will get different matrices for each language.

To detect the language just use all those matrices and use the probabilities as a score, let say that in English you'd get this probabilities:

t->h = 0.3 h->e = .2

and in the Spanish matrix you'd get that

t->h = 0.01 h->e = .3

The word 'the', using the English matrix, would give you a score of 0.3+0.2 = 0.5 and using the Spanish one: 0.01+0.3 = 0.31

The English matrix wins so that has to be English.
```

`reducer()` Maakt een matrix met van 28 bij 28 (ons alfabet is 28 lang) en vult per combinatie in hoevaak die voorkomt.

In [7]:
def reducer(c):
    """
    When a combination occurs, update matrix in the right place
    
    @param c: list of combinations found in text
    @return : matrix with combination frequencies
    """
    matrix = numpy.zeros((28, 28))
    for combination in c:
        left, right = combination[0], combination[-1]
        try:
            matrix[abc.index(left)][abc.index(right)] += 1 # find right place in matrix and add 1
        except:
            # double check: if one of the letters isnt in the alphabet skip this one and continue
            continue 
    
    return matrix

`reducer_chance_matrix()` gebruikt de door `reducer()` gemaakte matrix om een matrix te maken met de kans dat een combinatie voorkomt. `reducer_chance_matrix()` en `reducer()` worden gebruikt in de trainings fase.

In [8]:
def reducer_chance_matrix(m):
    """
    Convert a frequence matrix to a chance matrix
    
    @param m: frequencie matrix
    @return : chance matrix
    """
    total = numpy.sum(m) # total letter combinations analysed
    
    with numpy.nditer(m, op_flags=['readwrite']) as it:
        for x in it:
            if x != 0:
                x[...] = round(x / total * 100, 4) # chance x to its chance that it occurs
   
    return m

`reducer_final()` is een functie voor de validatie. Deze functie voorspelt met behulp van de kans matrices van `reducer_chance_matrix()` of een zin Nederlands of Engels is. Voor een specifieke uitleg over hoe het in zn werking gaat kan je de comments in de onderstaande functie lezen.

In [9]:
def reducer_final(eng_m, nl_m, c):
    """
    method: https://stackoverflow.com/a/18197422
    
    @param eng_m: english trained matrix numpy array
    @param nl_m: dutch trained matrix numpy array
    @param c: list van combinaties die voorkomen per zin
    """
    nl_count = 0    # houdt bij hoeveel nederlandse zinnen er zijn gevonden
    eng_count = 0   # houdt bij hoveel engelse zinnen er zijn gevonden
    for line in c:  # loopt door de tekst heen per line
        # De matrices zijn gevuld met kansen. Per combinatie die in een zin voorkomt
        # pakt het algoritme de bijbehorende kans en telt die bij de totale kans dat een
        # zin nederlands of engels is op. De taal met het hoogste getal aan het einde 
        # wordt als waarheid genomen.
        eng = 0   
        nl = 0
        for combination in line:
            left, right = abc.index(combination[0]), abc.index(combination[-1]) # vind juiste index voor matrices
            oem = eng_m[left][right]    # On English Matrix. De kans dat de combinatie voorkomt in het engels
            odm = nl_m[left][right]     # On Dutch Matrix. De kans dat de combinatie voorkomt in het engels
            eng += oem  # tel de kans op bij het totaal (voor sepcifieke zin)
            nl += odm
        
        # er is nu een complete zin gecheckt -> kijk welk getal groter is en tel één bij de telling op
        # Een gelijkspel zal zeer waarschijnlijk niet voorkomen omdat de kansen op 3 decimalen zijn afgerond
        if eng > nl:
            eng_count += 1
        else:
            nl_count += 1
        
        #reset waardes voor zin en ga door naar de volgende zin
        eng, nl = 0, 0  
    
    return nl_count, eng_count

### train_matrix.py
Hier worden de matrices getraind, en worden ze opgeslagen als een .npy bestand.
Met behulp van een paart `print()` statements laten we zien dat de gewenste resultaten worden behaald.

In [10]:
LANGUAGES = ['eng', 'nl']


def train_matrix(txt, lang):
    """
    Calls right functions to make and save a chance matrix
    
    @param  txt: list of combinations found in text
    @param lang: Language configuration
    """
    matrix = numpy.zeros((28, 28))
    for line in txt:
        m = reducer(line)
        matrix = numpy.add(matrix, m)
    
    chance_m = reducer_chance_matrix(matrix)

#     numpy.save("trained_matrix/" + lang + "_trained", chance_m)

`train()`

In [11]:
def train():
    """
    Train function. Uses outputs from a function as inputs to the next to produce two trained matrices.
    """

    for x in LANGUAGES:
        path = "train_" + x + "/train_data"
        
        lines_list = mapper_get_lines(path)
        updated_text = mapper_replace_punct(lines_list, leestekens)
        
        combi_list = filter_training(updated_text)
        
        print(f'Demonstratie mapper functies.\nBefore: {lines_list[0]} \nAfter mapper.py: {updated_text[0]} \n')
        print(f'Demonstratie filter functies. \nDit is hoe de combinaties van de vorige zin eruit zien:\n{combi_list[0:50]}\n\n\n')

        train_matrix(combi_list, x)

train()

Demonstratie mapper functies.
Before: ﻿The Project Gutenberg eBook of Metamorphosis, by Franz Kafka 
After mapper.py: ﻿the_project_gutenberg_ebook_of_metamorphosis%_by_franz_kafka 

Demonstratie filter functies. 
Dit is hoe de combinaties van de vorige zin eruit zien:
[['th'], ['he'], ['e_'], ['_p'], ['pr'], ['ro'], ['oj'], ['je'], ['ec'], ['ct'], ['t_'], ['_g'], ['gu'], ['ut'], ['te'], ['en'], ['nb'], ['be'], ['er'], ['rg'], ['g_'], ['_e'], ['eb'], ['bo'], ['oo'], ['ok'], ['k_'], ['_o'], ['of'], ['f_'], ['_m'], ['me'], ['et'], ['ta'], ['am'], ['mo'], ['or'], ['rp'], ['ph'], ['ho'], ['os'], ['si'], ['is'], ['s%'], ['_b'], ['by'], ['y_'], ['_f'], ['fr'], ['ra']]



Demonstratie mapper functies.
Before: The Project Gutenberg EBook of Alleen op de Wereld, by Hector Malot 
After mapper.py: the_project_gutenberg_ebook_of_alleen_op_de_wereld%_by_hector_malot 

Demonstratie filter functies. 
Dit is hoe de combinaties van de vorige zin eruit zien:
[['th'], ['he'], ['e_'], ['_p'], ['pr'], ['ro'

### main.py
Dit is de main functie, hier wordt alles in beweging gezet en uitgevoerd, ook wordt de uitkomst hier weergegeven.
Ook wordt hier weergegeven hoe de zin eruit ziet nadat alle spaties en leestekens weggehaald zijn, en daarna worden alle combinaties van die zin weergegeven.

In [12]:
def main():
    """
    Main function. Uses outputs from a function as inputs to the next.
    """

    lines_list = mapper_get_lines("validation")
    updated_text = mapper_replace_punct(lines_list, leestekens)

    combi_list = filter_validation(updated_text)

    nl, eng = numpy.load('trained_matrix/nl_trained.npy'), numpy.load('trained_matrix/eng_trained.npy')
    result = reducer_final(eng, nl, combi_list)
    
    print(f'Demonstratie mapper functies.\nBefore: {lines_list[0]} \nAfter mapper.py: {updated_text[0]} \n')
    print(f'Demonstratie filter functies. \nDit is hoe de combinaties van de vorige zin eruit zien:\n{combi_list[0]}\n\n\n')
    
    print(f'In deze tekst zijn {result[0]} Nederlandse zinnen gevonden, en {result[1]} Engelse zinnen.')

main()

Demonstratie mapper functies.
Before: time be generously rewarded by NASA.  Young descended those final few 
After mapper.py: time_be_generously_rewarded_by_nasa%__young_descended_those_final_few 

Demonstratie filter functies. 
Dit is hoe de combinaties van de vorige zin eruit zien:
['ti', 'im', 'me', 'e_', '_b', 'be', 'e_', '_g', 'ge', 'en', 'ne', 'er', 'ro', 'ou', 'us', 'sl', 'ly', 'y_', '_r', 're', 'ew', 'wa', 'ar', 'rd', 'de', 'ed', 'd_', '_b', 'by', 'y_', '_n', 'na', 'as', 'sa', 'a%', '__', '_y', 'yo', 'ou', 'un', 'ng', 'g_', '_d', 'de', 'es', 'sc', 'ce', 'en', 'nd', 'de', 'ed', 'd_', '_t', 'th', 'ho', 'os', 'se', 'e_', '_f', 'fi', 'in', 'na', 'al', 'l_', '_f', 'fe', 'ew']



In deze tekst zijn 74 Nederlandse zinnen gevonden, en 117 Engelse zinnen.


## Resultaat
Het streven was om in de validatie tesks 73 Nederlandse en 119 Engelse zinnen te herkennen. Ons resultaat laat zien dat onze code niet exact alle zinnen goed kan indelen, maar hij zit wel heel erg in de buurt van een 100% score. We zijn zelf tevreden met het resultaat van ons model.